In [148]:
# Import libraries
import pandas as pd
import numpy as np

In [149]:
# Upload users (influencers) dataset
df = pd.read_csv('influence_marketing_reco/datasets/influenceDataBrands.csv')
df

,gucci,Unnamed: 1,Welcome to the world of Gucci\non.gucci.com/Gucci2023,https://www.instagram.com/p/Cq0PiClsB11/,"248,092 likes",Harry Styles wears custom Gucci looks during his ‘Love on Tour’ concerts around the world.\n\n#HarryStyles,Unnamed: 6
0,gucci,NaN,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqxquUEMZ2k/,"41,519 likes","To craft the Gucci Horsebit 1955, the House’s ...",NaN
1,gucci,NaN,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqvvOTMsvS6/,"63,333 likes","Not just a bag, but a piece of history. The Gu...",NaN
2,gucci,NaN,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqu_GKPsCQr/,"37,633 likes","As one of the House’s signature bags, the Gucc...",NaN
3,gucci,NaN,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqr3_tjspzK/,"26,516 likes",From a lucky bunny to talking about self-expre...,NaN
4,gucci,NaN,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqp8Y24MD83/,"32,024 likes",Global Brand Ambassador Hanni from the Korean ...,NaN
...,...,...,...,...,...,...,...
1014,wholesome_for.you,NaN,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2Cve4qfIq/,376 likes,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,NaN
1015,wholesome_for.you,NaN,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CtI3KxOz/,878 likes,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,NaN
1016,wholesome_for.you,NaN,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CqpDqcqF/,971 likes,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,NaN
1017,wholesome_for.you,NaN,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/CqzddbbK1jL/,709 likes,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.\n.,NaN


In [150]:
df = df.rename(columns={'gucci': 'brand_name', 'Welcome to the world of Gucci\non.gucci.com/Gucci2023': 'brand_description', 'https://www.instagram.com/p/Cq0PiClsB11/': 'Brand_post_link', '248,092 likes': 'brand_likes', 'Harry Styles wears custom Gucci looks during his ‘Love on Tour’ concerts around the world.\n\n#HarryStyles': 'brand_post_description'})

In [151]:
df = df.drop(['Unnamed: 1', 'Unnamed: 6'], axis=1)

In [152]:
df['all_descriptions'] = df['brand_description'].str.cat(df['brand_post_description'], sep=' ')

In [153]:
from googletrans import Translator
from langdetect import detect

translator = Translator()
descripciones = []

count = 0

for index, row in df.iterrows():
    text = row['all_descriptions']
    count = count+1
    print(count)
    if text:
        try:
            language = detect(text)
            if language != 'en':
                translated = translator.translate(text, src=language, dest="en")
                descripcion = str(translated.text)
                descripciones.append(descripcion)
            else:
                descripciones.append(text)
        except:
            # if there is an error in translation, we can leave the original text
            descripciones.append(text)
    else:
        descripciones.append(text)

df['descriptions_translated'] = descripciones

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [154]:
df['brand_likes'] = df['brand_likes'].str.replace(',', '')

In [155]:
df.head(293)

,brand_name,brand_description,Brand_post_link,brand_likes,brand_post_description,all_descriptions,descriptions_translated
0,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqxquUEMZ2k/,41519 likes,"To craft the Gucci Horsebit 1955, the House’s ...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
1,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqvvOTMsvS6/,63333 likes,"Not just a bag, but a piece of history. The Gu...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
2,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqu_GKPsCQr/,37633 likes,"As one of the House’s signature bags, the Gucc...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
3,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqr3_tjspzK/,26516 likes,From a lucky bunny to talking about self-expre...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
4,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqp8Y24MD83/,32024 likes,Global Brand Ambassador Hanni from the Korean ...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
...,...,...,...,...,...,...,...
288,marsglobal,"Mars, Incorporated\nFamily-owned. Future-facin...",https://www.instagram.com/p/Cl884JgsXnW/,NaN,Heard of Zac Efron? The actor and global explo...,"Mars, Incorporated\nFamily-owned. Future-facin...","Mars, Incorporated\nFamily-owned. Future-facin..."
289,ferrerorocherbr,Ferrero Rocher Br\nDesde sua fundação no norte...,https://www.instagram.com/p/CqQjI-QgNyS/,142769 likes,Vim te ajudar a ganhar seu @ferrerorocherbr fa...,Ferrero Rocher Br\nDesde sua fundação no norte...,Ferrero Rocher Br\nSince its foundation in nor...
290,ferrerorocherbr,Ferrero Rocher Br\nDesde sua fundação no norte...,https://www.instagram.com/p/CqEf3u3g3vR/,83449 likes,A vida fica melhor quando encontramos momentos...,Ferrero Rocher Br\nDesde sua fundação no norte...,Ferrero Rocher Br\nSince its foundation in nor...
291,ferrerorocherbr,Ferrero Rocher Br\nDesde sua fundação no norte...,https://www.instagram.com/p/Cp0geAdvFBn/,867 likes,"Nessa Páscoa, viva momentos que valem ouro. Pr...",Ferrero Rocher Br\nDesde sua fundação no norte...,Ferrero Rocher Br\nSince its foundation in nor...


In [156]:
import pandas as pd
import re

df['brand_likes'] = df['brand_likes'].astype(str)

# Extracting numbers from column 'likes' and replacing non-numeric values with 0
df['brand_likes'] = df['brand_likes'].apply(lambda x: float(re.findall('\d+\.?\d*', x.replace(',', ''))[0]) if len(re.findall('\d+\.?\d*', x.replace(',', ''))) > 0 else 0)

# Displaying the modified DataFrame
df

,brand_name,brand_description,Brand_post_link,brand_likes,brand_post_description,all_descriptions,descriptions_translated
0,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqxquUEMZ2k/,41519.0,"To craft the Gucci Horsebit 1955, the House’s ...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
1,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqvvOTMsvS6/,63333.0,"Not just a bag, but a piece of history. The Gu...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
2,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqu_GKPsCQr/,37633.0,"As one of the House’s signature bags, the Gucc...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
3,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqr3_tjspzK/,26516.0,From a lucky bunny to talking about self-expre...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
4,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqp8Y24MD83/,32024.0,Global Brand Ambassador Hanni from the Korean ...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...
...,...,...,...,...,...,...,...
1014,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2Cve4qfIq/,376.0,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...
1015,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CtI3KxOz/,878.0,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...
1016,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CqpDqcqF/,971.0,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...
1017,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/CqzddbbK1jL/,709.0,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...


In [157]:
df['brand_name'] = df['brand_name'].astype(str)
df['brand_description'] = df['brand_description'].astype(str)
df['Brand_post_link'] = df['Brand_post_link'].astype(str)
df['brand_likes'] = df['brand_likes'].astype(int)
df['brand_post_description'] = df['brand_post_description'].astype(str)
df['all_descriptions'] = df['all_descriptions'].astype(str)
df['descriptions_translated'] = df['descriptions_translated'].astype(str)

In [158]:
import re

def get_hashtags(text):
    if pd.isnull(text):
        return []
    hashtags = re.findall(r'#\w+', text)
    # Remove the '#' symbol from each hashtag, convert to lowercase, and store only the word in the 'has' list
    has = [hashtag[1:].lower() for hashtag in hashtags]
    return has


In [159]:
has = []

for i in range(len(df['brand_post_description'])):
    print(df['brand_post_description'][i])
    has.append(get_hashtags(df['brand_post_description'][i]))

df['hashtags'] = has

To craft the Gucci Horsebit 1955, the House’s artisans construct and stitch together the contemporary silhouette, finishing it with the emblematic hardware that harnesses Gucci’s 102-year heritage. Discover more at the link in bio. ⁣
⁣
#GucciHorsebit1955
Not just a bag, but a piece of history. The Gucci Horsebit 1955 handbag progresses the narrative of a specific archival code – the double ring and bar that was presented in 1955, two years after the original hardware appeared in the House’s collections on a pair of loafers, becoming an iconic Gucci symbol and furthering its link to the equestrian world. Discover more at the link in bio.

#GucciHorsebit1955
As one of the House’s signature bags, the Gucci Horsebit 1955 bridges eras through the story it tells and the elegant strength of its design—a testament to the exceptional materials and craftsmanship, as well as the mythmaking that have always distinguished the House. Discover more at the link in bio.

#GucciHorsebit1955
From a lucky

In [160]:
df

,brand_name,brand_description,Brand_post_link,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags
0,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqxquUEMZ2k/,41519,"To craft the Gucci Horsebit 1955, the House’s ...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...,[guccihorsebit1955]
1,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/CqvvOTMsvS6/,63333,"Not just a bag, but a piece of history. The Gu...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...,[guccihorsebit1955]
2,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqu_GKPsCQr/,37633,"As one of the House’s signature bags, the Gucc...",Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...,[guccihorsebit1955]
3,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqr3_tjspzK/,26516,From a lucky bunny to talking about self-expre...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...,"[guccihorsebit1955, hanni]"
4,gucci,Welcome to the world of Gucci\non.gucci.com/Gu...,https://www.instagram.com/p/Cqp8Y24MD83/,32024,Global Brand Ambassador Hanni from the Korean ...,Welcome to the world of Gucci\non.gucci.com/Gu...,Welcome to the world of Gucci\non.gucci.com/Gu...,"[guccihorsebit1955, hanni]"
...,...,...,...,...,...,...,...,...
1014,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2Cve4qfIq/,376,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[]
1015,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CtI3KxOz/,878,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[]
1016,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/Cq2CqpDqcqF/,971,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[]
1017,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,https://www.instagram.com/p/CqzddbbK1jL/,709,Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.\n.,wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[]


In [161]:
# group by user_name and aggregate the other columns
df_grouped = df.groupby('brand_name').agg({
    'brand_name': 'first',
    'brand_description': 'first',
    'brand_likes': 'mean',
    'brand_post_description': lambda x: ','.join(x),
    'all_descriptions': lambda x: ','.join(x),
    'descriptions_translated': lambda x: ','.join(x),
    'hashtags': lambda x: ','.join([','.join(tags) for tags in x])

})

In [162]:
df_grouped['hashtags'] = df_grouped['hashtags'].apply(lambda x: [word.strip() for word in x.split(',') if len(word.strip()) > 0])

In [163]:
df_grouped

,brand_name,brand_description,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags
brand_name,,,,,,,
abcnetwork,abcnetwork,ABC\nAmerica’s Network.\nabc.com,1871.40,Who will make Top 24!? Tune in to #AmericanIdo...,ABC\nAmerica’s Network.\nabc.com Who will make...,ABC\nAmerica’s Network.\nabc.com Who will make...,"[americanidol, americanidol, americanidol, the..."
acerojoaquintorres,acerojoaquintorres,Joaquin Torres\nArchitectural designer\nA-cero...,1504.80,"Esta es, sin duda, una de las viviendas más gr...",Joaquin Torres\nArchitectural designer\nA-cero...,Joaquin Torres\nArchitectural Designer\nSteel\...,"[sitevisit, luxuryhomes, acerojoaquintorres, a..."
adidas,adidas,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas,155061.90,HIM 📸 @jharden13\n\n#HardenVol7 #adidasBasketb...,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas...,Adidas\n#Impossibleisnoting\nlinktr.ee/adidas ...,"[hardenvol7, adidasbasketball, impossibleisnot..."
airfrance,airfrance,Air France\n✈️ Welcome on board!\nPost your be...,12396.00,Getting ready for departure 😎\nWhich destinati...,Air France\n✈️ Welcome on board!\nPost your be...,Air France\n✈️ Welcome on board!\nPost your be...,"[airfrance, airplane, avgeek, aviation, travel..."
amazon,amazon,Amazon\nRetail company\nCurrently manifesting ...,1851.95,"📍Okinawa, Japan 📍Murano, Italy 📍Mackinac Islan...",Amazon\nRetail company\nCurrently manifesting ...,Amazon\nRetail company\nCurrently manifesting ...,"[blackisremarkable, itsonprime, blackisremarka..."
...,...,...,...,...,...,...,...
virginatlantic,virginatlantic,Virgin Atlantic\nWe’ve always championed indiv...,7758.00,"Our iconic Vivienne Westwood uniforms, designe...",Virgin Atlantic\nWe’ve always championed indiv...,Virgin Atlantic\nWe’ve always championed indiv...,"[seetheworlddifferently, seetheworlddifferentl..."
volkswagen,volkswagen,Volkswagen\nCars\nWelcome to our official chan...,9510.10,World premiere of our new ID. 2all.\n\n#concep...,Volkswagen\nCars\nWelcome to our official chan...,Volkswagen\nCars\nWelcome to our official chan...,"[conceptcar, worldpremiere, vwid2all, vwforthe..."
wholesome_for.you,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,744.25,"Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,Ta...",wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[]


In [164]:
df = df_grouped

In [165]:
import os

folder_path = 'influence_marketing_reco/datasets/categories'

files = [os.path.splitext(f)[0] for f in os.listdir(folder_path)]

print(files)

['art_design', 'belleza', 'business', 'cars_motorcycles', 'culture', 'education', 'fashion', 'finance_investing', 'fitness_sport', 'food_cooking', 'gaming', 'humor_memes', 'luxury_lifestyle', 'music', 'nature', 'parenting_family', 'pets', 'politics', 'realEstate', 'science', 'social', 'technology', 'travel']


In [166]:
import os

folder_path = 'influence_marketing_reco/datasets/categories'
lists_dict = {}

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        list_name = os.path.splitext(filename)[0]
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line.strip() for line in f.readlines()]
            lists_dict[list_name] = lines

print(lists_dict)

for key in lists_dict.keys():
    print(key)

{'art_design': ['humanities', 'printmaking', 'performing arts', 'music', 'sculpture', 'fine art', 'artwork', 'poetry', 'painting', 'crafts', 'drawing', 'impressionism', 'aesthetics', 'paint', 'film', 'artistry', 'ceramics', 'decoupage', 'miniate', 'photography', 'stipple', 'romanticism', 'culture', 'portray', 'museum', 'artistic creation', 'artist', 'artistic production', 'fauvism', 'science', 'artistic', 'expressionism', 'taxidermy', 'cubism', 'commerce', 'creativity', 'puppetry', 'entertainment', 'quick', 'creative', 'social', 'skills', 'painter', 'creation', 'iconography', 'styles', 'law', 'craft', 'combat', 'dance', 'visual arts', 'self', 'dadaism', 'pop art', 'sports', 'repaint', 'buster', 'triptych', 'education', 'history', 'fresco', 'literature', 'fighting', 'sculpt', 'cultural', 'kitsch', 'techniques', 'model', 'design', 'emotion', 'sciences', 'genre', 'etch', 'arts', 'hand', 'craftsman', 'depict', 'distemper', 'media', 'playwriting', 'creative arts', 'artistic expression', 'co

In [167]:
max_categories = []

# loop through each row of the dataframe and calculate the category with the highest count
for index, row in df.iterrows():
    # create a dictionary to store the counts for each category
    counts = {key: 0 for key in lists_dict.keys()}
    # loop through each word in the text and increment the counts if the word matches a category
    for word in row['descriptions_translated'].split():
        for category, words in lists_dict.items():
            if word.lower() in words:
                counts[category] += 1
    # find the category with the highest count
    max_count = max(counts.values())
    max_category = [category for category, count in counts.items() if count == max_count][0]
    # append the max category to the list
    max_categories.append(max_category)

# add the max category column to the dataframe
df['category'] = max_categories

In [168]:
df

,brand_name,brand_description,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags,category
brand_name,,,,,,,,
abcnetwork,abcnetwork,ABC\nAmerica’s Network.\nabc.com,1871.40,Who will make Top 24!? Tune in to #AmericanIdo...,ABC\nAmerica’s Network.\nabc.com Who will make...,ABC\nAmerica’s Network.\nabc.com Who will make...,"[americanidol, americanidol, americanidol, the...",social
acerojoaquintorres,acerojoaquintorres,Joaquin Torres\nArchitectural designer\nA-cero...,1504.80,"Esta es, sin duda, una de las viviendas más gr...",Joaquin Torres\nArchitectural designer\nA-cero...,Joaquin Torres\nArchitectural Designer\nSteel\...,"[sitevisit, luxuryhomes, acerojoaquintorres, a...",social
adidas,adidas,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas,155061.90,HIM 📸 @jharden13\n\n#HardenVol7 #adidasBasketb...,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas...,Adidas\n#Impossibleisnoting\nlinktr.ee/adidas ...,"[hardenvol7, adidasbasketball, impossibleisnot...",social
airfrance,airfrance,Air France\n✈️ Welcome on board!\nPost your be...,12396.00,Getting ready for departure 😎\nWhich destinati...,Air France\n✈️ Welcome on board!\nPost your be...,Air France\n✈️ Welcome on board!\nPost your be...,"[airfrance, airplane, avgeek, aviation, travel...",social
amazon,amazon,Amazon\nRetail company\nCurrently manifesting ...,1851.95,"📍Okinawa, Japan 📍Murano, Italy 📍Mackinac Islan...",Amazon\nRetail company\nCurrently manifesting ...,Amazon\nRetail company\nCurrently manifesting ...,"[blackisremarkable, itsonprime, blackisremarka...",social
...,...,...,...,...,...,...,...,...
virginatlantic,virginatlantic,Virgin Atlantic\nWe’ve always championed indiv...,7758.00,"Our iconic Vivienne Westwood uniforms, designe...",Virgin Atlantic\nWe’ve always championed indiv...,Virgin Atlantic\nWe’ve always championed indiv...,"[seetheworlddifferently, seetheworlddifferentl...",social
volkswagen,volkswagen,Volkswagen\nCars\nWelcome to our official chan...,9510.10,World premiere of our new ID. 2all.\n\n#concep...,Volkswagen\nCars\nWelcome to our official chan...,Volkswagen\nCars\nWelcome to our official chan...,"[conceptcar, worldpremiere, vwid2all, vwforthe...",parenting_family
wholesome_for.you,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,744.25,"Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,Ta...",wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[],social


In [169]:
from googletrans import Translator, LANGUAGES

# define a function to detect the language of a list of words and translate them
def translate_words(words):
    if not words:
        return []
    try:
        # initialize the translator
        translator = Translator()
        # detect the language of the input words using the detect() method
        source_lang = translator.detect(' '.join(words)).lang
        print(source_lang)
        if source_lang != 'en':
            # translate each word using the translator
            translated_words = [translator.translate(word, src=source_lang, dest="en").text for word in words]
            print(translated_words)
            return translated_words
        else:
            return words
    except:
        return words

df['hashtags'] = df['hashtags'].fillna("")

# apply the function to the 'hashtags' column
df['hashtags_translated'] = df['hashtags'].apply(translate_words)

en
es
['Sitevisit', 'Luxuryhomes', 'Steeljoaquintorres', 'Architecture', 'architecture', 'Project', 'project', 'Process', 'Design', 'design', 'Luxury', 'Luxurylifestyle', 'Madrid', 'Luxury Villas', 'CONSTRUCTION', 'construction', 'luxury', 'Villasdelujo', 'Realestate', 'real estate', 'Luxury Real Estate', 'Easter', 'Andalusia', 'getaways', 'tourism', 'Spain', 'Steeljoaquintorres', 'Joaquintorres', 'architect', 'Lifestyle', 'living', 'passion', 'Luxuryhomes', 'Steeljoaquintorres', 'Architecture', 'architecture', 'Project', 'project', 'Process', 'Design', 'design', 'Luxury', 'Luxurylifestyle', 'Madrid', 'Luxury Villas', 'CONSTRUCTION', 'construction', 'luxury', 'Villasdelujo', 'Realestate', 'real estate', 'Luxury Real Estate', 'architecture', 'Steeljoaquintorres', 'Architecture', 'Windows', 'windows', 'light', 'Project', 'project', 'design', 'Design', 'architecture and design', 'Archilover', 'architect', 'architecture', 'Architecture', 'Project', 'project', 'Houses', 'households', 'Villa

In [170]:
df.head(50)

,brand_name,brand_description,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags,category,hashtags_translated
brand_name,,,,,,,,,
abcnetwork,abcnetwork,ABC\nAmerica’s Network.\nabc.com,1871.400000,Who will make Top 24!? Tune in to #AmericanIdo...,ABC\nAmerica’s Network.\nabc.com Who will make...,ABC\nAmerica’s Network.\nabc.com Who will make...,"[americanidol, americanidol, americanidol, the...",social,"[americanidol, americanidol, americanidol, the..."
acerojoaquintorres,acerojoaquintorres,Joaquin Torres\nArchitectural designer\nA-cero...,1504.800000,"Esta es, sin duda, una de las viviendas más gr...",Joaquin Torres\nArchitectural designer\nA-cero...,Joaquin Torres\nArchitectural Designer\nSteel\...,"[sitevisit, luxuryhomes, acerojoaquintorres, a...",social,"[Sitevisit, Luxuryhomes, Steeljoaquintorres, A..."
adidas,adidas,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas,155061.900000,HIM 📸 @jharden13\n\n#HardenVol7 #adidasBasketb...,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas...,Adidas\n#Impossibleisnoting\nlinktr.ee/adidas ...,"[hardenvol7, adidasbasketball, impossibleisnot...",social,"[hardenvol7, adidasbasketball, impossibleisnot..."
airfrance,airfrance,Air France\n✈️ Welcome on board!\nPost your be...,12396.000000,Getting ready for departure 😎\nWhich destinati...,Air France\n✈️ Welcome on board!\nPost your be...,Air France\n✈️ Welcome on board!\nPost your be...,"[airfrance, airplane, avgeek, aviation, travel...",social,"[airfrance, airplane, avgeek, aviation, travel..."
amazon,amazon,Amazon\nRetail company\nCurrently manifesting ...,1851.950000,"📍Okinawa, Japan 📍Murano, Italy 📍Mackinac Islan...",Amazon\nRetail company\nCurrently manifesting ...,Amazon\nRetail company\nCurrently manifesting ...,"[blackisremarkable, itsonprime, blackisremarka...",social,"[blackisremarkable, itsonprime, blackisremarka..."
amctheatres,amctheatres,AMC Theatres\nWe Make Movies Better. 🎟🍿🎬\nsimp...,1018.300000,Get your tickets for #GotGVol3 and experience ...,AMC Theatres\nWe Make Movies Better. 🎟🍿🎬\nsimp...,AMC Theatres\nWe Make Movies Better. 🎟🍿🎬\nsimp...,"[gotgvol3, filmedforimax, fastx, thecovenant]",social,"[gotgvol3, filmedforimax, fastx, thecovenant]"
americanair,americanair,American Airlines\nThe official Instagram acco...,8461.400000,It’s scientifically proven that your feelings ...,American Airlines\nThe official Instagram acco...,American Airlines\nThe official Instagram acco...,"[flagshipvalor, medalofhonor, avgeeks, aateam,...",parenting_family,"[flagshipvalor, medalofhonor, avgeeks, aateam,..."
amnesty,amnesty,Amnesty International 🌎\nNon-profit organisati...,3215.000000,Progress happens when we come together to dema...,Amnesty International 🌎\nNon-profit organisati...,Amnesty International 🌎\nNon-profit organisati...,"[protecttheprotest, protecttheprotest, endisra...",social,"[protecttheprotest, protecttheprotest, endisra..."
ana.japan,ana.japan,ANA✈️\nワクワクで満たされる世界を ✈️\nTikTok👇\nana.ms/ANATT,10169.300000,貨物専用機✈️✨#allnipponairways #ana #freighter #貨物専...,ANA✈️\nワクワクで満たされる世界を ✈️\nTikTok👇\nana.ms/ANATT...,ANA✈️\nワクワクで満たされる世界を ✈️\nTikTok👇\nana.ms/ANATT...,"[allnipponairways, ana, freighter, 貨物専用機, フレイタ...",travel,"[Allnipponairways, ANA, FREIGHTER, Cargo -only..."


In [171]:
# create a column with random numbers from 10000 to 100000
df['followers'] = np.random.randint(low=100000, high=20000000, size=len(df))

# print the DataFrame
df

,brand_name,brand_description,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags,category,hashtags_translated,followers
brand_name,,,,,,,,,,
abcnetwork,abcnetwork,ABC\nAmerica’s Network.\nabc.com,1871.40,Who will make Top 24!? Tune in to #AmericanIdo...,ABC\nAmerica’s Network.\nabc.com Who will make...,ABC\nAmerica’s Network.\nabc.com Who will make...,"[americanidol, americanidol, americanidol, the...",social,"[americanidol, americanidol, americanidol, the...",4172512
acerojoaquintorres,acerojoaquintorres,Joaquin Torres\nArchitectural designer\nA-cero...,1504.80,"Esta es, sin duda, una de las viviendas más gr...",Joaquin Torres\nArchitectural designer\nA-cero...,Joaquin Torres\nArchitectural Designer\nSteel\...,"[sitevisit, luxuryhomes, acerojoaquintorres, a...",social,"[Sitevisit, Luxuryhomes, Steeljoaquintorres, A...",4228417
adidas,adidas,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas,155061.90,HIM 📸 @jharden13\n\n#HardenVol7 #adidasBasketb...,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas...,Adidas\n#Impossibleisnoting\nlinktr.ee/adidas ...,"[hardenvol7, adidasbasketball, impossibleisnot...",social,"[hardenvol7, adidasbasketball, impossibleisnot...",2041166
airfrance,airfrance,Air France\n✈️ Welcome on board!\nPost your be...,12396.00,Getting ready for departure 😎\nWhich destinati...,Air France\n✈️ Welcome on board!\nPost your be...,Air France\n✈️ Welcome on board!\nPost your be...,"[airfrance, airplane, avgeek, aviation, travel...",social,"[airfrance, airplane, avgeek, aviation, travel...",11110475
amazon,amazon,Amazon\nRetail company\nCurrently manifesting ...,1851.95,"📍Okinawa, Japan 📍Murano, Italy 📍Mackinac Islan...",Amazon\nRetail company\nCurrently manifesting ...,Amazon\nRetail company\nCurrently manifesting ...,"[blackisremarkable, itsonprime, blackisremarka...",social,"[blackisremarkable, itsonprime, blackisremarka...",1932665
...,...,...,...,...,...,...,...,...,...,...
virginatlantic,virginatlantic,Virgin Atlantic\nWe’ve always championed indiv...,7758.00,"Our iconic Vivienne Westwood uniforms, designe...",Virgin Atlantic\nWe’ve always championed indiv...,Virgin Atlantic\nWe’ve always championed indiv...,"[seetheworlddifferently, seetheworlddifferentl...",social,"[seetheworlddifferently, seetheworlddifferentl...",6308799
volkswagen,volkswagen,Volkswagen\nCars\nWelcome to our official chan...,9510.10,World premiere of our new ID. 2all.\n\n#concep...,Volkswagen\nCars\nWelcome to our official chan...,Volkswagen\nCars\nWelcome to our official chan...,"[conceptcar, worldpremiere, vwid2all, vwforthe...",parenting_family,"[conceptcar, worldpremiere, vwid2all, vwforthe...",9416439
wholesome_for.you,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,744.25,"Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,Ta...",wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[],social,[],1786934


In [172]:
import random

# define the list of countries
countries = ['China', 'India', 'United States', 'Indonesia', 'Pakistan', 'Brazil', 'Nigeria', 'Bangladesh', 'Russia', 'Mexico', 'Japan', 'Ethiopia', 'Philippines', 'Egypt', 'Vietnam', 'DR Congo', 'Iran', 'Turkey', 'Germany', 'France', 'Thailand', 'United Kingdom', 'Italy', 'Tanzania', 'South Africa', 'Myanmar', 'Kenya', 'South Korea', 'Colombia', 'Spain']

# generate random countries from the list and add them to the DataFrame
df['location'] = [random.choice(countries) for _ in range(len(df))]

In [173]:
df

,brand_name,brand_description,brand_likes,brand_post_description,all_descriptions,descriptions_translated,hashtags,category,hashtags_translated,followers,location
brand_name,,,,,,,,,,,
abcnetwork,abcnetwork,ABC\nAmerica’s Network.\nabc.com,1871.40,Who will make Top 24!? Tune in to #AmericanIdo...,ABC\nAmerica’s Network.\nabc.com Who will make...,ABC\nAmerica’s Network.\nabc.com Who will make...,"[americanidol, americanidol, americanidol, the...",social,"[americanidol, americanidol, americanidol, the...",4172512,Pakistan
acerojoaquintorres,acerojoaquintorres,Joaquin Torres\nArchitectural designer\nA-cero...,1504.80,"Esta es, sin duda, una de las viviendas más gr...",Joaquin Torres\nArchitectural designer\nA-cero...,Joaquin Torres\nArchitectural Designer\nSteel\...,"[sitevisit, luxuryhomes, acerojoaquintorres, a...",social,"[Sitevisit, Luxuryhomes, Steeljoaquintorres, A...",4228417,Turkey
adidas,adidas,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas,155061.90,HIM 📸 @jharden13\n\n#HardenVol7 #adidasBasketb...,adidas\n#ImpossibleIsNothing\nlinktr.ee/adidas...,Adidas\n#Impossibleisnoting\nlinktr.ee/adidas ...,"[hardenvol7, adidasbasketball, impossibleisnot...",social,"[hardenvol7, adidasbasketball, impossibleisnot...",2041166,United Kingdom
airfrance,airfrance,Air France\n✈️ Welcome on board!\nPost your be...,12396.00,Getting ready for departure 😎\nWhich destinati...,Air France\n✈️ Welcome on board!\nPost your be...,Air France\n✈️ Welcome on board!\nPost your be...,"[airfrance, airplane, avgeek, aviation, travel...",social,"[airfrance, airplane, avgeek, aviation, travel...",11110475,Tanzania
amazon,amazon,Amazon\nRetail company\nCurrently manifesting ...,1851.95,"📍Okinawa, Japan 📍Murano, Italy 📍Mackinac Islan...",Amazon\nRetail company\nCurrently manifesting ...,Amazon\nRetail company\nCurrently manifesting ...,"[blackisremarkable, itsonprime, blackisremarka...",social,"[blackisremarkable, itsonprime, blackisremarka...",1932665,DR Congo
...,...,...,...,...,...,...,...,...,...,...,...
virginatlantic,virginatlantic,Virgin Atlantic\nWe’ve always championed indiv...,7758.00,"Our iconic Vivienne Westwood uniforms, designe...",Virgin Atlantic\nWe’ve always championed indiv...,Virgin Atlantic\nWe’ve always championed indiv...,"[seetheworlddifferently, seetheworlddifferentl...",social,"[seetheworlddifferently, seetheworlddifferentl...",6308799,South Africa
volkswagen,volkswagen,Volkswagen\nCars\nWelcome to our official chan...,9510.10,World premiere of our new ID. 2all.\n\n#concep...,Volkswagen\nCars\nWelcome to our official chan...,Volkswagen\nCars\nWelcome to our official chan...,"[conceptcar, worldpremiere, vwid2all, vwforthe...",parenting_family,"[conceptcar, worldpremiere, vwid2all, vwforthe...",9416439,South Korea
wholesome_for.you,wholesome_for.you,wholesome memes\n💝 I try to make your relation...,744.25,"Tag someone ❤️\n-\n\n.\nDM for Credit\n.\n.,Ta...",wholesome memes\n💝 I try to make your relation...,wholesome memes\n💝 I try to make your relation...,[],social,[],1786934,Bangladesh


In [ ]:
df.to_csv('brands_df.csv')